In [7]:
%load_ext autoreload
%autoreload 2

from dataloader import FakenNewsNet
import tensorflow as tf
from model import Net
from spektral.data import DisjointLoader
from tensorflow import keras
from tqdm import tqdm
import numpy as np

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_accuracy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def train_step(model, loss_fn, optimizer, inputs, target):

    with tf.GradientTape() as tape:
        predictions, _, _ = model(inputs[:2])  # drop edge features from inputs
        loss = loss_fn(target, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(binary_accuracy(target, predictions))
    return loss, acc

In [3]:
def random_split(data, train_pct=0.75, train_epochs=5, seed=None):
    """ returns two DataLoaders, resp. for train and test set 
        according to the input split percentage """
    if seed is not None:
        np.random.default_rng(seed)

    np.random.shuffle(data)
    n = len(data)
    train_size = int(n*train_pct)

    train_set = data[:train_size]
    test_set = data[train_size:]

    train_loader = DisjointLoader(train_set, batch_size=1, epochs=train_epochs)
    test_loader = DisjointLoader(test_set, batch_size=1, epochs=1)

    return train_loader, test_loader

In [6]:
def evaluate(train_loader, test_loader):
    model = Net(num_classes=2)
    loss_fn = BinaryCrossentropy(from_logits=True)
    optimizer = Adam(learning_rate=0.001)

    epoch = step = 0
    train_results = []

    for inputs, target in tqdm(train_loader):
        step += 1
        loss, acc = train_step(model, loss_fn, optimizer, inputs, target)
        train_results.append((loss, acc))

        if step == train_loader.steps_per_epoch:
            step = 0
            epoch += 1
            print("Ep. {} - Loss: {}. Acc: {}".format(epoch, *np.mean(train_results, 0)))
            train_results = []
    
    test_results = []

    for inputs, target in test_loader:
        predictions, _, _ = model(inputs[:2], training=False)
        test_results.append(
            (
                loss_fn(target, predictions),
                tf.reduce_mean(categorical_accuracy(target, predictions)),
            )
        )
    
    print("Done. Test loss: {}. Test acc: {}".format(*np.mean(test_results, 0)))

    return train_results, test_results

In [9]:
data = FakenNewsNet("./dataset/FakeNewsNet/politifact/")  

crossval_results = []
n_splits = 1

for i in range(n_splits):
    print(f"Experiment {i+1} of {n_splits}")
    train_loader, test_loader = random_split(data, train_epochs=3)

    train_results, test_results = evaluate(train_loader, test_loader)

    crossval_results.append((train_results, test_results))

Experiment 1 of 1


502it [02:45,  2.89it/s]

Ep. 1 - Loss: 0.7530835270881653. Acc: 0.5


1004it [05:40,  2.66it/s]

Ep. 2 - Loss: 0.7521498799324036. Acc: 0.5


1506it [08:27,  2.97it/s]

Ep. 3 - Loss: 0.7520725727081299. Acc: 0.5


KeyboardInterrupt: 